In [1]:
import random

from deap import base, creator, tools, algorithms

In [14]:
N_QUEENS = 8
random.seed(42)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
 
toolbox = base.Toolbox()
toolbox.register("permutation", random.sample, range(N_QUEENS), N_QUEENS)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.permutation)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
  
    
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)

In [15]:
def visualize(queens):
    print(queens)
    print("  " + "|".join([str(i) for i in range(len(queens))]) + "|")
    for row in range(len(queens)):
        col = queens.index(row)
        print(str(row) + "| "*(col) + "|x" + "| "*(len(queens)-col))

In [16]:
ind = list(toolbox.individual())
visualize(ind)

[1, 0, 5, 2, 7, 6, 4, 3]
  0|1|2|3|4|5|6|7|
0| |x| | | | | | | 
1|x| | | | | | | | 
2| | | |x| | | | | 
3| | | | | | | |x| 
4| | | | | | |x| | 
5| | |x| | | | | | 
6| | | | | |x| | | 
7| | | | |x| | | | 


One complete solution. Each index of list corresponds to one columns. Value on that index is a number number of row.
The queens are therefore always in different columns (indices in list are unique) and also in different row (random.sample takes N_QUEENS WITHOUT replacement, so all values in this list are unique).
Only think to be taken care of is the diagonals.

In [17]:
# number of endangered queens
def evaluate(queens):
    conflicts = 0
    for col1 in range(len(queens)):
        for col2 in range(col1+1,len(queens)):
            row1 = queens[col1]
            row2 = queens[col2]
            if abs(row2-row1) == abs(col1-col2):
                conflicts += 1
    return conflicts,

assert evaluate([0,1,2]) == (3,)
assert evaluate([1,3,0,2]) == (0,)
toolbox.register("evaluate", evaluate)

In [18]:
pop = toolbox.population(n=200)

NGEN = 50
CXPB = 0.5
MUTPB = 0.2

solutions, gen_info = algorithms.eaSimple(pop, toolbox, cxpb=CXPB, 
                                          mutpb=MUTPB, ngen=NGEN, 
                                          verbose=True)
print(f"There are {len(solutions)} solutions.")
unique_solutions = [eval(r) for r in set([str(s) for s in solutions])]
print(f"There are {len(unique_solutions)} unique solutions.")
correct_solutions = [s for s in unique_solutions if evaluate(s)==(0,)]
print(f"{len(correct_solutions)} of them are correct.")
print("Let's visualize a few:")
for s in correct_solutions[:3]:
    visualize(s)

gen	nevals
0  	200   
1  	102   
2  	127   
3  	119   
4  	114   
5  	115   
6  	118   
7  	124   
8  	119   
9  	115   
10 	112   
11 	131   
12 	115   
13 	119   
14 	118   
15 	128   
16 	118   
17 	127   
18 	138   
19 	116   
20 	112   
21 	105   
22 	116   
23 	125   
24 	134   
25 	130   
26 	110   
27 	130   
28 	110   
29 	122   
30 	103   
31 	122   
32 	118   
33 	117   
34 	109   
35 	123   
36 	112   
37 	123   
38 	133   
39 	117   
40 	112   
41 	122   
42 	124   
43 	116   
44 	113   
45 	127   
46 	121   
47 	120   
48 	133   
49 	121   
50 	113   
There are 200 solutions.
There are 40 unique solutions.
2 of them are correct.
Let's visualize a few:
[2, 5, 7, 0, 3, 6, 4, 1]
  0|1|2|3|4|5|6|7|
0| | | |x| | | | | 
1| | | | | | | |x| 
2|x| | | | | | | | 
3| | | | |x| | | | 
4| | | | | | |x| | 
5| |x| | | | | | | 
6| | | | | |x| | | 
7| | |x| | | | | | 
[2, 5, 7, 0, 4, 6, 1, 3]
  0|1|2|3|4|5|6|7|
0| | | |x| | | | | 
1| | | | | | |x| | 
2|x| | | | | | | | 
3| | | | | | | |x|

In [19]:
# with different algorithm

# generate new population
pop = toolbox.population(n=200)

for g in range(NGEN):
    # Select and clone the next generation individuals
    offspring = map(toolbox.clone, toolbox.select(pop, len(pop)))
 
    # Apply crossover and mutation on the offspring
    offspring = algorithms.varAnd(offspring, toolbox, CXPB, MUTPB)
 
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
 
    # The population is entirely replaced by the offspring
    pop[:] = offspring

print(f"There are {len(pop)} solutions.")
unique_solutions = [eval(r) for r in set([str(s) for s in pop])]
print(f"There are {len(unique_solutions)} unique solutions.")
correct_solutions = [s for s in unique_solutions if evaluate(s)==(0,)]
print(f"{len(correct_solutions)} of them are correct.")
print("Let's visualize a few:")
for s in correct_solutions[:3]:
    visualize(s)

There are 200 solutions.
There are 32 unique solutions.
2 of them are correct.
Let's visualize a few:
[4, 7, 3, 0, 6, 1, 5, 2]
  0|1|2|3|4|5|6|7|
0| | | |x| | | | | 
1| | | | | |x| | | 
2| | | | | | | |x| 
3| | |x| | | | | | 
4|x| | | | | | | | 
5| | | | | | |x| | 
6| | | | |x| | | | 
7| |x| | | | | | | 
[2, 5, 7, 1, 3, 0, 6, 4]
  0|1|2|3|4|5|6|7|
0| | | | | |x| | | 
1| | | |x| | | | | 
2|x| | | | | | | | 
3| | | | |x| | | | 
4| | | | | | | |x| 
5| |x| | | | | | | 
6| | | | | | |x| | 
7| | |x| | | | | | 
